# Stream Chat with flex flow

**Learning Objectives** - Upon completing this tutorial, you should be able to:

- Write LLM application using class based flex flow.
- Use AzureOpenAIModelConfiguration as class init parameter.
- Use prompty to stream completions.
- Convert the application into a flow and batch run against multi lines of data.
- Use classed base flow to evaluate the main flow and learn how to do aggregation.

## 0. Install dependent packages

In [1]:
%%capture --no-stderr
%pip install -r ./requirements.txt

## 1. Trace your application with promptflow

Assume we already have a python program, which leverage prompty.

In [2]:
with open("flow.py") as fin:
    print(fin.read())

from pathlib import Path

from promptflow.tracing import trace
from promptflow.core import AzureOpenAIModelConfiguration, Prompty

BASE_DIR = Path(__file__).absolute().parent


class ChatFlow:
    def __init__(self, model_config: AzureOpenAIModelConfiguration):
        self.model_config = model_config

    @trace
    def __call__(
        self, question: str = "What is ChatGPT?", chat_history: list = None
    ) -> str:
        """Flow entry function."""

        chat_history = chat_history or []

        prompty = Prompty.load(
            source=BASE_DIR / "chat.prompty",
            model={"configuration": self.model_config},
        )

        # output is a generator of string as prompty enabled stream parameter
        output = prompty(question=question, chat_history=chat_history)

        return output


if __name__ == "__main__":
    from promptflow.tracing import start_trace

    start_trace()
    config = AzureOpenAIModelConfiguration(
        connection="open_ai_connection", a

When `stream=true` is configured in the parameters of a prompt whose output format is text, promptflow sdk will return a generator type, which item is the content of each chunk.

Reference openai doc on how to do it using plain python code: [how_to_stream_completions](https://cookbook.openai.com/examples/how_to_stream_completions)

In [3]:
with open("chat.prompty") as fin:
    print(fin.read())

---
name: Stream Chat
description: Chat with stream enabled.
model:
  api: chat
  configuration:
    type: azure_openai
    azure_deployment: gpt-35-turbo
  parameters:
    temperature: 0.2
    stream: true
inputs: 
  question:
    type: string
  chat_history:
    type: list
sample: sample.json
---

system:
You are a helpful assistant.

{% for item in chat_history %}
{{item.role}}:
{{item.content}}
{% endfor %}

user:
{{question}}




### Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

Above prompty uses connection `open_ai_connection` inside, we need to set up the connection if we haven't added it before. After created, it's stored in local db and can be used in any flow.

Prepare your Azure Open AI resource follow this [instruction](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) and get your `api_key` if you don't have one.

In [4]:
from promptflow.client import PFClient
from promptflow.connections import AzureOpenAIConnection, OpenAIConnection

# client can help manage your runs and connections.
pf = PFClient()
try:
    conn_name = "open_ai_connection"
    conn = pf.connections.get(name=conn_name)
    print("using existing connection")
except:
    # Follow https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal to create an Azure Open AI resource.
    connection = AzureOpenAIConnection(
        name=conn_name,
        api_key="<your_AOAI_key>",
        api_base="<your_AOAI_endpoint>",
        api_type="azure",
    )

    # use this if you have an existing OpenAI account
    # connection = OpenAIConnection(
    #     name=conn_name,
    #     api_key="<user-input>",
    # )

    conn = pf.connections.create_or_update(connection)
    print("successfully created connection")

print(conn)

using existing connection
name: open_ai_connection
module: promptflow.connections
created_date: '2023-11-20T09:50:43.325090'
last_modified_date: '2024-04-24T11:01:25.463786'
type: azure_open_ai
api_key: '******'
api_base: https://gpt-test-eus.openai.azure.com/
api_type: azure
api_version: '2024-02-01'
auth_mode: key



### Visualize trace by using start_trace

Note we add `@trace` in the `my_llm_tool` function, re-run below cell will collect a trace in trace UI.

In [5]:
from promptflow.tracing import start_trace
from promptflow.core import AzureOpenAIModelConfiguration

from flow import ChatFlow

# create a chatFlow obj with connection
config = AzureOpenAIModelConfiguration(
    connection="open_ai_connection", azure_deployment="gpt-35-turbo"
)
chat_flow = ChatFlow(config)

# start a trace session, and print a url for user to check trace
start_trace()

# run the flow as function, which will be recorded in the trace
result = chat_flow(question="What is ChatGPT? Please explain with consise statement")
# note the type is generator object as we enabled stream in prompty
result

Prompt flow service has started...
You can view the traces from local: http://localhost:61746/v1.0/ui/traces/?#collection=chat-stream


<generator object traced_generator at 0x00000249DE134660>

In [6]:
# print result in stream manner
for r in result:
    print(result, end="")

<generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_generator at 0x00000249DE134660><generator object traced_genera

### Eval the result 

In [7]:
%load_ext autoreload
%autoreload 2

import paths  # add the code_quality module to the path
from check_list import EvalFlow

eval_flow = EvalFlow(config)
# evaluate answer agains a set of statement
eval_result = eval_flow(
    answer=result.answer,
    statements={
        "correctness": "It contains a detailed explanation of ChatGPT.",
        "consise": "It is a consise statement.",
    },
)
eval_result

AttributeError: 'generator' object has no attribute 'answer'

## 2. Batch run the function as flow with multi-line data


### Batch run with a data file (with multiple lines of test data)


In [ ]:
from promptflow.client import PFClient

pf = PFClient()

In [ ]:
data = "./data.jsonl"  # path to the data file
# create run with the flow function and data
base_run = pf.run(
    flow=chat_flow,
    data=data,
    column_mapping={
        "question": "${data.question}",
    },
    stream=True,
)

In [ ]:
details = pf.get_details(base_run)
details.head(10)

## 3. Evaluate your flow
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which usually using LLM assert the produced output matches certain expectation. 

### Run evaluation on the previous batch run
The **base_run** is the batch run we completed in step 2 above, for web-classification flow with "data.jsonl" as input.

In [ ]:
eval_run = pf.run(
    flow=eval_flow,
    data="./data.jsonl",  # path to the data file
    run=base_run,  # specify base_run as the run you want to evaluate
    column_mapping={
        "answer": "${run.outputs.answer}",
        "statements": "${data.statements}",
    },
    stream=True,
)

In [ ]:
details = pf.get_details(eval_run)
details.head(10)

In [ ]:
import json

metrics = pf.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

In [ ]:
pf.visualize([base_run, eval_run])

## Next steps

By now you've successfully run your chat flow and did evaluation on it. That's great!

You can check out more examples:
- [Stream Chat](https://github.com/microsoft/promptflow/tree/main/examples/flex-flows/chat-stream): demonstrates how to create a chatbot that runs in streaming mode.